In [162]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [163]:
def fix_JSON(json_message=None):
    result = None
    try:        
        result = json.loads(json_message)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(e.message.split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        json_message = list(json_message)
        json_message[idx_to_replace] = ' '
        new_message = ''.join(json_message)     
        return fix_JSON(json_message=new_message)
    return result

In [164]:
explainers = BeautifulSoup(requests.get("http://theconversation.com/au/factcheck").text)
headlines = explainers.find_all('h2')
results = list()

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [165]:
for headline in headlines:
    thisline = dict()
    try:
        link = "http://theconversation.com" + headline.a['href']
        thisline['headline'] = headline.text
        thisline['content'] = requests.get(link).text
        soup = BeautifulSoup(thisline['content'])
        thisline['link'] = link
        meta = soup.find(attrs={'name' : "news_keywords"})
        thisline['keywords'] = meta['content']
    except:
        pass
    results.append(thisline)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [170]:
results[1]['keywords']

'Coalition, Election factcheck 2016, Federal election 2016, General practice, Health, Labor Party, Liberal Party, Medicare, Medicare rebates, bulk-billing, factcheck'

In [171]:
abcexplainers = BeautifulSoup(requests.get('http://www.abc.net.au/news/factcheck/zombies/').text)

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [186]:
links = abcexplainers.find_all("a")
links = [link for link in links if "Fact Check zombie" in link.text]

In [173]:
thisline = list()

In [179]:
link = links[0]

In [180]:
abcid = link['href'].split('/')[-1]
url = "https://content-api-govhack.abc-prod.net.au/v1/" + abcid
data = requests.get(url).text

In [181]:
jsondoc = json.loads(data)

JSONDecodeError: Invalid \escape: line 73 column 499 (char 9310)

In [159]:
thisline.append(jsondoc)

In [187]:
for link in links:
    abcid = link['href'].split('/')[-1]
    url = "https://content-api-govhack.abc-prod.net.au/v1/" + abcid
    data = requests.get(url).text
    try:
        jsondoc = json.loads(data.replace('\r\n', ''))
        thisline.append(jsondoc)
    except:
        print("That went horribly wrong")

That went horribly wrong
That went horribly wrong
That went horribly wrong


In [190]:
thisline[0].keys()

dict_keys(['shortTeaserTextPlain', 'title', 'rootNavigation', 'thumbnailLink', 'id', 'keywords', 'shortTeaserTitle', 'text', 'textPlain', 'lastPublishedDate', 'shortTeaserText', 'version', 'relatedItems', 'contentGenre', 'teaserTextPlain', 'firstPublishedDate', 'docType', 'teaserTitle', 'subject', 'teaserText', 'latitude', 'location', 'importance', 'canonicalUrl', 'longitude'])

In [177]:
df = pd.DataFrame(results)

In [192]:
df = df.append(thisline)

In [193]:
df.to_pickle('explainers.pickle')